In [1]:
import pandas as pd
import numpy as np

In [2]:
# specify the input file name and output file name
habitat_matrix = f'../data/raw_data/larval_habitat.csv'
species_file = f'../data/raw_data/species_data.csv'
habitat_classification_key = f'../data/raw_data/habitat_classification_key.csv'

# reads in the input file as a .csv
habitat_df = pd.read_csv(habitat_matrix)
species_df = pd.read_csv(species_file)

In [3]:
# removes any hidden carriage returns and spaces
def CarriageRemover( pandas_datafram ):
    pandas_datafram = pandas_datafram.replace({r'\r\n': ''}, regex=True)
    pandas_datafram = pandas_datafram.replace({r'\s': ''}, regex=True)
    pandas_datafram = pandas_datafram.replace({r'\n': ''}, regex=True)
    return pandas_datafram

In [4]:
habitat_df = CarriageRemover(habitat_df)
species_df = CarriageRemover(species_df)

In [5]:
# merge both datasets
habitat_species_df = pd.merge(species_df, habitat_df, left_on=['Record_ID'], right_on=['Record_ID'])

In [6]:
# remove columns without genus and species level data
sp_col = ['Pub_ID', 'Subfamily', 'Tribe', 'Subspecies', '"Form"', 'Record_ID', 'Notes_x', 'Notes_y']
habitat_species_df = habitat_species_df.drop(columns=sp_col)

In [7]:
# changes the NaN designation in the 'Subgenus' column to 'None'
#subgenus_column = habitat_species_df['Subgenus'].where(pd.notnull(habitat_species_df['Subgenus']), 'None')
habitat_species_df['Subgenus'] = habitat_species_df['Subgenus'].where(pd.notnull(habitat_species_df['Subgenus']), 'None')

habitat_species_df = habitat_species_df[habitat_species_df['Species'] != 'sp.']
habitat_species_df = habitat_species_df[habitat_species_df['Species'] != 'Sp.']

habitat_species_df = habitat_species_df[habitat_species_df['Presence-Absence'] == 'Presence']

habitat_species_df = habitat_species_df.reset_index(drop=True)

print(habitat_species_df, habitat_species_df.columns)
habitat_species_df['Genus_Species'] = habitat_species_df['Genus'] + '_' + habitat_species_df['Species']

#habitat_species_df.to_csv('../data/intermediate_data/intermed_hab_sp.csv', index=False)

          Genus            Subgenus      Species Presence-Absence  \
0     Anopheles              Cellia      farauti         Presence   
1     Anopheles              Cellia  punctulatus         Presence   
2     Anopheles              Cellia    koliensis         Presence   
3     Anopheles              Cellia       lungae         Presence   
4     Anopheles              Cellia     nataliae         Presence   
...         ...                 ...          ...              ...   
3541      Culex  SubgenusUncertain2      romeroi         Presence   
3542   Mansonia            Mansonia     flaveola         Presence   
3543      Aedes        Ochlerotatus     tortilis         Presence   
3544      Aedes           Stegomyia      aegypti         Presence   
3545      Culex               Culex   bahamensis         Presence   

     Shaded (Shaded/Semishaded/Unshaded) Saltwater (Yes/No)  \
0                                    NaN                NaN   
1                                    NaN     

In [8]:
# map habitat descriptions to abbreviation

habitat_abbv = pd.read_csv(habitat_classification_key)
habitat_abbv = CarriageRemover(habitat_abbv)
habitat_abbv = habitat_abbv.dropna()

habitat_columns = [f'Habitat_{a}' for a in range(1, 51)]

for i in habitat_columns:
    curr_hab = habitat_species_df[i].tolist()
    hab_abbv = []
    #print(curr_hab)
    for j in curr_hab:
        #print(habitat_abbv.loc[habitat_abbv['Unique_Habitat'] == j, 'Habitat_Classifier'].iloc[0])
        try:
            hab_abbv.append(habitat_abbv.loc[habitat_abbv['Unique_Habitat'] == j, 'Habitat_Classifier'].iloc[0])
        except:
            hab_abbv.append('None')

    habitat_species_df[i] = hab_abbv

habitat_species_df = habitat_species_df.dropna(axis=1, how='all')

In [9]:
# make relationship matrix

unique_hab_list = habitat_abbv.Habitat_Classifier.unique().tolist()
habitat_species_relationship = pd.DataFrame(columns=['Genus_Species'] + unique_hab_list)
#print(habitat_species_relationship)
total_habitat_types = len(unique_hab_list)

species_list = []

for i, j in enumerate(habitat_columns):
    curr_hab = habitat_species_df[j].tolist()

    for g, c in enumerate(curr_hab):

        if c == 'None':
            pass
        else:
            prop_spp = habitat_species_df.loc[g, 'Genus_Species']

            if prop_spp not in species_list:
                habs = [0] * total_habitat_types

                habs[unique_hab_list.index(c)] = 1

                habs.insert(0, prop_spp)

                new_row = pd.Series({key: value for key, value in zip(habitat_species_relationship.columns, habs)})
                habitat_species_relationship = pd.concat([habitat_species_relationship, new_row.to_frame().T], ignore_index=True)
                species_list.append(prop_spp)

            else:
                try:
                    indx = habitat_species_relationship.loc[habitat_species_relationship['Genus_Species'] == prop_spp].index[0]
                    #print(indx)
                    habitat_species_relationship.loc[indx, c] += 1
                except:
                    print(prop_spp)
                    print(habitat_species_relationship['Genus_Species'].tolist())


nan
['Anopheles_farauti', 'Anopheles_punctulatus', 'Anopheles_koliensis', 'Anopheles_nataliae', 'Anopheles_solomonis', 'Uranotaenia_painei', 'Uranotaenia_wysockii', 'Uranotaenia_sexaueri', 'Uranotaenia_civinskii', 'Uranotaenia_lateralis', 'Uranotaenia_solomonis', 'Culex_pervigilans', 'Culex_pacificus', 'Culex_iyengari', 'Culex_australicus', 'Culex_atriceps', 'Culex_kesseli', 'Culex_roseni', 'Culex_omani', 'Anopheles_atropos', 'Anopheles_barberi', 'Anopheles_crucians', 'Anopheles_earlei', 'Anopheles_freeborni', 'Anopheles_georgianus', 'Anopheles_occidentalis', 'Anopheles_franciscanus', 'Anopheles_punctipennis', 'Anopheles_walkeri', 'Toxorhynchites_rutilus', 'Wyeomyia_mitchellii', 'Wyeomyia_smithii', 'Wyeomyia_vanduzeei', 'Uranotaenia_anhydor', 'Uranotaenia_lowii', 'Uranotaenia_sapphirina', 'Culiseta_alaskaensis', 'Culiseta_impatiens', 'Culiseta_incidens', 'Culiseta_inornata', 'Culiseta_particeps', 'Culiseta_morsitans', 'Culiseta_melanura', 'Orthopodomyia_alba', 'Orthopodomyia_signifera'

In [10]:
# calculate some summary stats
habitat_species_relationship['SUM'] = np.sum(habitat_species_relationship.drop(columns=['Genus_Species']).values, axis=1)
habitat_species_relationship['UNIQUE_SUM'] = np.count_nonzero(habitat_species_relationship.drop(columns=['Genus_Species', 'SUM']).values, axis=1)
habitat_species_relationship['PROPORTION'] = habitat_species_relationship['UNIQUE_SUM'] / total_habitat_types

habitat_species_relationship = habitat_species_relationship.dropna()
print(habitat_species_relationship)

                   Genus_Species AC ALG ATR BAM BOG BP BRK BWP CEM  ... SWB  \
0              Anopheles_farauti  3   0   0   0   0  0   0   1   0  ...   0   
1          Anopheles_punctulatus  0   0   2   0   0  0   0   0   0  ...   0   
2            Anopheles_koliensis  0   0   1   0   0  0   0   0   0  ...   0   
3             Anopheles_nataliae  0   0   0   0   0  0   0   0   0  ...   0   
4            Anopheles_solomonis  0   0   0   0   0  0   1   0   0  ...   0   
...                          ... ..  ..  ..  ..  .. ..  ..  ..  ..  ...  ..   
1994  Trichoprosopon_vonplesseni  0   0   0   1   0  0   0   0   0  ...   0   
1995    Shannoniana_schedocyclia  0   0   0   1   0  0   0   0   0  ...   0   
1996               Culex_marksae  0   0   0   0   0  0   0   0   0  ...   0   
1997           Aedes_formosensis  0   0   0   1   0  0   0   0   0  ...   0   
1998              Culex_usquatus  1   0   0   0   0  0   0   0   0  ...   0   

     SWG TH TSTM VEG VLC VTR SUM UNIQUE_SUM PROPORT

In [25]:
# minimu number of species in a habitat type to be included
MINIMUM_SPECIES_UNIQUE = 10
MINIMUM_SPECIES_NON_UNIQUE = 0

In [26]:

habitat_species_df = habitat_species_relationship.drop(columns=['SUM', 'UNIQUE_SUM', 'PROPORTION'])

# Drop columns with n or less observations
drop_cols_non_unique = []
for i in habitat_species_df.columns:
    print(i, len(habitat_species_df[habitat_species_df[i] == 1].index.tolist()))
    if len(habitat_species_df[habitat_species_df[i] == 1].index.tolist()) <= MINIMUM_SPECIES_NON_UNIQUE:
        drop_cols_non_unique.append(i)

habitat_species_df = habitat_species_df.drop(columns=drop_cols_non_unique)

print(habitat_species_df.columns)


# species ID nodes
species_nodes = list(habitat_species_df.index)
print(len(species_nodes))
# habitat nodes -- removes the 'Record_ID' column at the first position
habitat_nodes = list(habitat_species_df.columns)
print(len(habitat_nodes))
print(habitat_nodes)
# retrieves on the values from the matrix for looping through all values to make tuples out of column/row relationships
val_matrix = habitat_species_df.values
print(len(val_matrix))
print(val_matrix)

Genus_Species 0
AC 149
ALG 13
ATR 90
BAM 216
BOG 14
BP 17
BRK 4
BWP 27
CEM 11
CNL 20
COR 19
CRH 78
CST 7
CV 4
DAM 17
DRN 62
DTH 141
FH 82
FNG 4
FP 20
FPP 92
GDN 14
GP 417
GPA 2
GPB 17
GPT 242
HGW 15
IC 54
LA 317
LK 39
LGN 28
LKA 2
LKB 1
LKM 23
LMS 6
MAQ 7
ML 54
MNR 2
MR 9
MRB 14
MRH 87
PLT 8
PNT 6
PO 142
POA 1
POM 21
POT 1
PST 3
PTH 25
RC 87
RD 55
RIV 50
RIVD 1
RIVF 1
RIVM 40
RIVRM 3
RIVT 1
RP 245
RPB 13
SEP 65
SHL 13
SND 4
SNW 21
SOL 1
SPG 45
STM 182
STMD 10
STMF 4
STMM 131
STMRM 29
STMT 19
SW 210
SWB 2
SWG 9
TH 343
TSTM 2
VEG 4
VLC 9
VTR 29
Index(['AC', 'ALG', 'ATR', 'BAM', 'BOG', 'BP', 'BRK', 'BWP', 'CEM', 'CNL',
       'COR', 'CRH', 'CST', 'CV', 'DAM', 'DRN', 'DTH', 'FH', 'FNG', 'FP',
       'FPP', 'GDN', 'GP', 'GPA', 'GPB', 'GPT', 'HGW', 'IC', 'LA', 'LK', 'LGN',
       'LKA', 'LKB', 'LKM', 'LMS', 'MAQ', 'ML', 'MNR', 'MR', 'MRB', 'MRH',
       'PLT', 'PNT', 'PO', 'POA', 'POM', 'POT', 'PST', 'PTH', 'RC', 'RD',
       'RIV', 'RIVD', 'RIVF', 'RIVM', 'RIVRM', 'RIVT', 'RP', 'RPB', 'SEP'

In [27]:
habitat_species_df

,AC,ALG,ATR,BAM,BOG,BP,BRK,BWP,CEM,CNL,...,STMRM,STMT,SW,SWB,SWG,TH,TSTM,VEG,VLC,VTR
0,3,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1995,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1997,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Removes all columns have that less than 10 species

cols_to_drop = []

# For each column, check if it is the only contributor to the row sum
for col in habitat_species_df.columns:
    temp_df = habitat_species_df.copy()
    temp_df[col] = 0
    if not (temp_df.sum(axis=1) == 0).any():
        cols_to_drop.append(col)

# Drop unnecessary columns
df_unique = habitat_species_df.drop(columns=cols_to_drop)

# Drop columns with n or less observations
drop_cols = []
for i in df_unique.columns:
    print(i, len(df_unique[df_unique[i] == 1].index.tolist()))
    if len(df_unique[df_unique[i] == 1].index.tolist()) <= MINIMUM_SPECIES_UNIQUE:
        drop_cols.append(i)

df_unique = df_unique.drop(columns=drop_cols)

print(df_unique.columns)

AC 149
ATR 90
BAM 216
BP 17
COR 19
CRH 78
CV 4
DRN 62
DTH 141
FH 82
FP 20
FPP 92
GP 417
GPB 17
GPT 242
HGW 15
IC 54
LA 317
LK 39
LGN 28
ML 54
MR 9
MRB 14
MRH 87
PLT 8
PO 142
POM 21
RC 87
RD 55
RIVM 40
RIVRM 3
RP 245
RPB 13
SEP 65
SNW 21
SPG 45
STM 182
STMD 10
STMF 4
STMM 131
STMRM 29
STMT 19
SW 210
TH 343
Index(['AC', 'ATR', 'BAM', 'BP', 'COR', 'CRH', 'DRN', 'DTH', 'FH', 'FP', 'FPP',
       'GP', 'GPB', 'GPT', 'HGW', 'IC', 'LA', 'LK', 'LGN', 'ML', 'MRB', 'MRH',
       'PO', 'POM', 'RC', 'RD', 'RIVM', 'RP', 'RPB', 'SEP', 'SNW', 'SPG',
       'STM', 'STMM', 'STMRM', 'STMT', 'SW', 'TH'],
      dtype='object')


In [29]:
# total remaining habitats
len(df_unique.columns)

38

In [30]:
# check to make sure all rows are unique
# species ID nodes
species_nodes_unique = list(df_unique.index)
print(len(species_nodes_unique))
# habitat nodes -- removes the 'Record_ID' column at the first position
habitat_nodes_unique = list(df_unique.columns)
print(len(habitat_nodes_unique))
print(habitat_nodes_unique)
# retrieves on the values from the matrix for looping through all values to make tuples out of column/row relationships
val_matrix_unique = df_unique.values
print(len(val_matrix_unique))
print(val_matrix_unique)


1998
38
['AC', 'ATR', 'BAM', 'BP', 'COR', 'CRH', 'DRN', 'DTH', 'FH', 'FP', 'FPP', 'GP', 'GPB', 'GPT', 'HGW', 'IC', 'LA', 'LK', 'LGN', 'ML', 'MRB', 'MRH', 'PO', 'POM', 'RC', 'RD', 'RIVM', 'RP', 'RPB', 'SEP', 'SNW', 'SPG', 'STM', 'STMM', 'STMRM', 'STMT', 'SW', 'TH']
1998
[[3 0 0 ... 0 1 0]
 [0 2 0 ... 0 0 0]
 [0 1 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [32]:
# dropped species
unique_indx = df_unique[df_unique.sum(axis=1) != 0].index.tolist()
non_unique_indx = habitat_species_df[habitat_species_df.sum(axis=1) != 0].index.tolist()

print(len(unique_indx), len(non_unique_indx), len(non_unique_indx) - len(unique_indx))
print(set(unique_indx)^set(non_unique_indx))

unique_habs = df_unique.columns.tolist()
non_unique_habs = habitat_species_df.columns.tolist()
print(set(unique_habs)^set(non_unique_habs))
print(len(set(unique_habs)^set(non_unique_habs)))
# export unique species df

1989 1998 9
{417, 513, 592, 611, 769, 851, 1035, 1276, 1537}
{'SWB', 'VEG', 'LKA', 'RIV', 'MAQ', 'POA', 'SHL', 'GDN', 'GPA', 'BRK', 'RIVD', 'ALG', 'LKB', 'PTH', 'CV', 'TSTM', 'PST', 'MR', 'MNR', 'SND', 'SWG', 'DAM', 'VLC', 'BWP', 'FNG', 'PLT', 'BOG', 'LKM', 'RIVF', 'SOL', 'CNL', 'CST', 'RIVT', 'RIVRM', 'STMF', 'STMD', 'POT', 'LMS', 'PNT', 'CEM', 'VTR'}
41


In [33]:
# output uncleaned df to .csv
habitat_species_relationship.to_csv('../data/cleaned_data/expert_habitat_species_matrix.csv', index=False)

# export the cleaned df
df_unique.to_csv('../data/cleaned_data/cleaned_unique_species_hab_matrix.csv')